In [2]:
# installing required packages for this notebook session
sc.install_pypi_package("haversine")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/f4/52/a13286844780c7b1740edbbee8a8f0524e2a6d51c068b59dda39a6a119f5/haversine-2.3.0-py2.py3-none-any.whl

In [5]:
traces = spark.read.parquet("s3://mobility-traces-sp/teste_minute_MO_15101/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
traces.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+----------+-----------+--------+------+----------+-----------+--------------------+
|id_avl_1|line_1|latitude_1|longitude_1|id_avl_2|line_2|latitude_2|longitude_2|    graph_id_minutes|
+--------+------+----------+-----------+--------+------+----------+-----------+--------------------+
|   58320| 34201| -23.47438| -46.714153|   55160| 33577|-23.465322| -46.721725|      18-56-PIRITUBA|
|   58123| 32953|-23.677788| -46.762158|   40457|  1242|-23.690227| -46.773953|  14-0-JARDIM ANGELA|
|   44116| 33128|-23.649757| -46.705398|   57768|  2236|-23.653923| -46.704765|    7-46-SANTO AMARO|
|   57704|  2196|-23.561885|  -46.51017|   42274|  1026|-23.564513| -46.507637|    14-41-ARICANDUVA|
|   23060|  1368|-23.505482|  -46.68894|   13178| 34128|-23.501038| -46.686545|         17-29-LIMAO|
|   57708|  1492|-23.676597| -46.643258|   57529|  1287|-23.682227| -46.652157|  7-37-CIDADE ADEMAR|
|    9840| 34404| -23.61417| -46.501255|   58486|  1631| -23.59842| -46.499913|      8-29-S

In [3]:
from haversine import haversine, Unit
import pyspark.sql.functions as F

def get_distance(lat1,lon1,lat2,lon2):
    coord_1 = (lat1,lon1)
    coord_2 = (lat2,lon2)

    distance = haversine(coord_1,coord_2,unit=Unit.METERS)
    
    return distance
    
get_distance_udf = F.udf(get_distance)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
# Analyzing 1 min + region 
df_1min = spark.read.parquet("s3://mobility-traces-sp/teste_minute_MO_15101/")
df_without_repeated = df_1min.filter("id_avl_1 != id_avl_2")

# save without repeated in s3

df_final = df_without_repeated.withColumn("distance",get_distance_udf(F.col("latitude_1"),F.col("longitude_1"),F.col("latitude_2"),F.col("longitude_2")))

df_final.filter("distance <= 100")\
    .groupby("id_avl_1","graph_id_minutes")\
    .agg(F.countDistinct("id_avl_2").alias("counter_buses_connections"))\
    .write.parquet("s3://mobility-traces-sp/teste_minute_MO_15101_count_vehicles/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
# Analyzing 1 min + region 
df_counts = spark.read.parquet("s3://mobility-traces-sp/teste_minute_MO_15101_count_vehicles/")
df_counts.show()
df_counts.agg(F.avg("counter_buses_connections")).show()

x = df_counts.approxQuantile("counter_buses_connections", [0.25,0.5,0.75], 0.0001)
print(x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+-------------------------+
|id_avl_1|    graph_id_minutes|counter_buses_connections|
+--------+--------------------+-------------------------+
|   11974|          20-0-LIMAO|                        4|
|    9209|       20-42-SANTANA|                       40|
|   47750| 18-22-CIDADE ADEMAR|                        1|
|   36880|             22-1-SE|                       79|
|   38997|12-54-JARDIM PAUL...|                       14|
|   57487|7-1-CIDADE TIRADE...|                       30|
|   37694|     22-21-JABAQUARA|                       34|
|   55116|      11-20-PIRITUBA|                       34|
|   36895|         15-43-MOEMA|                        5|
|   12398|      21-31-PIRITUBA|                       28|
|   40182| 10-26-CAPAO REDONDO|                       30|
|   11613|    12-3-SANTO AMARO|                       19|
|   37211|       15-44-SOCORRO|                        1|
|   13738|            13-59-SE|                        3|
|   11521|  18

In [51]:
# Analyzing 5 min + region 
df_5min = spark.read.parquet("s3://mobility-traces-sp/teste_minute_MO_15101_5min/")
df_without_repeated_5min = df_5min.filter("id_avl_1 != id_avl_2")

# save without repeated in s3

df_final_5min = df_without_repeated_5min.withColumn("distance",get_distance_udf(F.col("latitude_1"),F.col("longitude_1"),F.col("latitude_2"),F.col("longitude_2")))

df_final_5min.filter("distance <= 100")\
    .groupby("id_avl_1","graph_id")\
    .agg(F.countDistinct("id_avl_2").alias("counter_buses_connections"))\
    .write.parquet("s3://mobility-traces-sp/teste_5m_MO_15101_count_vehicles/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
# Analyzing 5 min + region 
df_5min_count = spark.read.parquet("s3://mobility-traces-sp/teste_5m_MO_15101_count_vehicles/")
print(df_5min_count.count())
df_5min_count.show()
df_5min_count.agg(F.avg("counter_buses_connections")).show()

results = df_5min_count.approxQuantile("counter_buses_connections", [0.25,0.5,0.75], 0.0001)
print(results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2853558
+--------+--------------------+-------------------------+
|id_avl_1|            graph_id|counter_buses_connections|
+--------+--------------------+-------------------------+
|   37688|          12-9-MOEMA|                       23|
|   34081|        7-2-PIRITUBA|                       58|
|   11315|      16-0-PINHEIROS|                       60|
|   11466|         13-7-GRAJAU|                       31|
|   42186|         13-8-CARRAO|                       49|
|   51281|        10-5-TATUAPE|                        8|
|   10931|        10-3-CAMBUCI|                        9|
|   42179|   10-1-CIDADE LIDER|                       30|
|   11113|13-10-JARDIM SAO ...|                       25|
|   35291|          17-11-BRAS|                        8|
|   57002|        9-11-TATUAPE|                       42|
|   39137|      8-5-ITAIM BIBI|                       37|
|   41665|    16-11-ITAIM BIBI|                       11|
|   39592|  14-2-CAPAO REDONDO|                       58|
|    9

In [ ]:
# Analyzing 15 seconds + without region 
# toke too much to process
df_15s = spark.read.parquet("s3://mobility-traces-sp/teste_MO_15101_15s_without-region/")
df_without_repeated_15s = df_15s.filter("id_avl_1 != id_avl_2")

# save without repeated in s3

df_final_15s = df_without_repeated_15s.withColumn("distance",get_distance_udf(F.col("latitude_1"),F.col("longitude_1"),F.col("latitude_2"),F.col("longitude_2")))

df_final_15s.filter("distance <= 100")\
    .groupby("id_avl_1","graph_id")\
    .agg(F.countDistinct("id_avl_2").alias("counter_buses_connections"))\
    .write.parquet("s3://mobility-traces-sp/teste_15s_MO_15101_count_vehicles/")

In [4]:
# Analyzing 15 seconds + with region 

df_15s_with_region = spark.read.parquet("s3://mobility-traces-sp/teste_MO_15101_15s_with-region/")
df_without_repeated_15s_with_region = df_15s_with_region.filter("id_avl_1 != id_avl_2")

# save without repeated in s3

df_final_15s_with_region = df_without_repeated_15s_with_region.withColumn("distance",get_distance_udf(F.col("latitude_1"),F.col("longitude_1"),F.col("latitude_2"),F.col("longitude_2")))

df_final_15s_with_region.filter("distance <= 100")\
    .groupby("id_avl_1","graph_id")\
    .agg(F.countDistinct("id_avl_2").alias("counter_buses_connections"))\
    .write.parquet("s3://mobility-traces-sp/teste_15s_with_region_MO_15101_count_vehicles/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Analyzing 15s + region 
df_15s_count_with_region = spark.read.parquet("s3://mobility-traces-sp/teste_15s_with_region_MO_15101_count_vehicles/")
df_15s_count_with_region.show()
df_15s_count_with_region.agg(F.avg("counter_buses_connections")).show()

results_15s_with_region = df_15s_count_with_region.approxQuantile("counter_buses_connections", [0.25,0.5,0.75], 0.0001)
print(results_15s_with_region)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+-------------------------+
|id_avl_1|           graph_id|counter_buses_connections|
+--------+-------------------+-------------------------+
|   11338| 11-142-SANTO AMARO|                        2|
|   36953|7-201-JARDIM ANGELA|                        2|
|   42149|     10-7-SAPOPEMBA|                        2|
|   55230|11-172-CACHOEIRINHA|                        9|
|   11727|19-219-CIDADE DUTRA|                        2|
|   35345|        8-4-TATUAPE|                        5|
|   40961|   15-167-PINHEIROS|                        1|
|   11698|         9-64-MOEMA|                        1|
|   40471| 14-173-SANTO AMARO|                        2|
|   56832|      16-88-TATUAPE|                        5|
|   30891|      15-51-JAGUARA|                        5|
|   47573|  9-100-CAMPO LIMPO|                        9|
|   56001|    13-145-ITAQUERA|                        6|
|   57402|    22-220-PEDREIRA|                        6|
|   38372|   8-153-VILA SONIA| 